Group 3 -Finance  Agentic Project

In [ ]:
!pip install --upgrade pip
!pip install yfinance transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 96.6 MB/s  0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import faiss
from transformers import pipeline
from sentence_transformers import SentenceTransformer

PlannerAgent - The agent begins by outlining a structured plan for analyzing a stock, including collecting one year of historical price
and volume data, computing key market trends, and generating an evaluation summary.
This step sets the foundation for methodical and reusable research logic

In [ ]:
class PlannerAgent:
    def plan_research(self, ticker):
        steps = [
            "Collect 1 year historical data for " + ticker,
            "Compute price and volume trends for " + ticker,
            "Generate short term outlook and evaluation summary for " + ticker
        ]
        return steps

# Test PlannerAgent for AAPL
planner = PlannerAgent()
plan = planner.plan_research("AAPL")
print("Planner Agent Output")
for s in plan:
    print("-", s)


Planner Agent Output
- Collect 1 year historical data for AAPL
- Compute price and volume trends for AAPL
- Generate short term outlook and evaluation summary for AAPL


2.	Data Acquisition (DataAgent)
Historical daily price data is fetched directly from Yahoo Finance's public API, focusing on recent 1-year trading activity. The data is cleaned and validated, particularly extracting the closing price to assess recent market behavior.


In [ ]:
class DataAgent:
    def get_yahoo_data(self, ticker):
        df = yf.download(ticker, period="1y", interval="1d", progress=False)
        if df is None or df.empty:
            print("Warning No data found for", ticker)
            return pd.DataFrame()
        df_tail = df.tail(10)
        if "Close" not in df_tail.columns or df_tail["Close"].dropna().empty:
            print("Warning No closing price data for", ticker)
            return pd.DataFrame()
        return df_tail

# Test DataAgent for AAPL
data_agent = DataAgent()
df = data_agent.get_yahoo_data("AAPL")
print("Data Agent Output (Tail 10 Days)")
print(df)


/tmp/ipython-input-614755891.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", interval="1d", progress=False)


Data Agent Output (Tail 10 Days)
Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2025-10-06  256.690002  259.070007  255.050003  257.989990  44664100
2025-10-07  256.480011  257.399994  255.429993  256.809998  31955800
2025-10-08  258.059998  258.519989  256.109985  256.519989  36496900
2025-10-09  254.039993  258.000000  253.139999  257.809998  38322000
2025-10-10  245.270004  256.380005  244.000000  254.940002  61999100
2025-10-13  247.660004  249.690002  245.559998  249.380005  38142900
2025-10-14  247.770004  248.850006  244.699997  246.600006  35478000
2025-10-15  249.339996  251.820007  247.470001  249.490005  33893600
2025-10-16  247.449997  249.039993  245.130005  248.250000  39777000
2025-10-17  252.289993  253.380005  247.270004  248.020004  48876500


3.	Market Trend Analysis (AnalysisAgent)
Using the acquired data, the agent calculates daily returns to deduce average price change and volatility,
thereby detecting whether the stock is trending upwards or downwards. This quantitative summary balances recent market dynamics.


In [ ]:
class AnalysisAgent:
    def analyze_market_trend(self, df):
        if df.empty:
            return "No Data", 0.0, 0.0

        # Handle MultiIndex columns from yfinance
        if isinstance(df.columns, pd.MultiIndex):
            try:
                # Select the Close column for the first ticker
                close_col = df["Close"].iloc[:, 0]
            except Exception as e:
                print("Error accessing Close column:", e)
                return "No Data", 0.0, 0.0
        else:
            if "Close" in df.columns:
                close_col = df["Close"]
            else:
                print("No Close column found")
                return "No Data", 0.0, 0.0

        daily_change = close_col.pct_change().dropna()
        if daily_change.empty:
            return "No Data", 0.0, 0.0

        change = daily_change.mean() * 100
        volatility = daily_change.std() * 100
        trend = "uptrend" if change > 0 else "downtrend"
        return trend, round(change, 2), round(volatility, 2)

# Test AnalysisAgent for AAPL
analysis_agent = AnalysisAgent()
trend, change, volatility = analysis_agent.analyze_market_trend(df)
print("Analysis Agent Output (Tail 10 Days)")
print("Trend:", trend)
print("Average Change:", change)
print("Volatility:", volatility)


Analysis Agent Output (Tail 10 Days)
Trend: downtrend
Average Change: -0.18
Volatility: 1.59


4.	Keyword Extraction for News (Helper Function)
To enhance decision-making, relevant keywords tied to the stock's ticker, industry, and sector are dynamically extracted from Yahoo Finance metadata.
These keywords enable focused searches for news articles related to the stock.


In [ ]:
import yfinance as yf

def get_related_keywords(ticker):
    t = yf.Ticker(ticker)
    info = t.info
    industry = info.get("industry")
    sector = info.get("sector")
    # Combine industry, sector, and ticker as search keywords
    keywords = []
    if ticker:
        keywords.append(ticker)
    if industry:
        keywords.append(industry)
    if sector:
        keywords.append(sector)
    return keywords

# Example
keywords = get_related_keywords("AAPL")
print("Keywords for news search:", keywords)


Keywords for news search: ['AAPL', 'Consumer Electronics', 'Technology']


5.	News Retrieval and Summarization (NewsAgent)
The agent fetches financial news headlines from multiple freely available RSS feeds
 (including Yahoo Finance, CNBC, and Reuters). Headlines are filtered by the keyword list
 to ensure topical relevance. The resulting brief news summary helps incorporate market sentiment and developments.


In [ ]:
import feedparser

class NewsAgent:
    def __init__(self, rss_urls=None):
        if rss_urls is None:
            rss_urls = [
                "https://finance.yahoo.com/rss/topstories",
                "https://www.cnbc.com/id/100003114/device/rss/rss.html",
                "https://www.reuters.com/rssFeed/technologyNews"
            ]
        self.rss_urls = rss_urls

    def fetch_news(self, ticker=None, max_articles=5):
        headlines = []
        keywords = get_related_keywords(ticker) if ticker else []

        for url in self.rss_urls:
            feed = feedparser.parse(url)
            for entry in feed.entries:
                headline = getattr(entry, "title", None)
                if headline:
                    # Check if any keyword appears in the headline
                    if any(k.lower() in headline.lower() for k in keywords):
                        headlines.append(headline)
                if len(headlines) >= max_articles:
                    break
            if len(headlines) >= max_articles:
                break
        return headlines

    def summarize_news(self, news_list):
        return " | ".join(news_list[:3]) if news_list else "No news available"

# Test for Apple
news_agent = NewsAgent()
news_headlines = news_agent.fetch_news("AAPL")
news_summary = news_agent.summarize_news(news_headlines)

print("\nFiltered News Agent Output (Industry and Sector-related Headlines for AAPL)")
print(news_summary)



Filtered News Agent Output (Industry and Sector-related Headlines for AAPL)
Alkami Technology (ALKT) Partners with HFC Union for New Mobile Banking Experience


6.	Final Recommendation Generation (SummaryAgent)
Utilizing a state-of-the-art natural language generation model (Google's Flan-T5),
the agent synthesizes data trends and recent news to produce a clear "buy" or "not buy" recommendation.
The output is concise, interpretable, and grounded only in available quantitative and textual data.


In [ ]:
class SummaryAgent:
    def __init__(self):
        self.llm = pipeline("text2text-generation", model="google/flan-t5-base")

    def generate_summary(self, ticker, trend, change, volatility, news_summary):
        prompt = (
            f"You are a financial assistant. You are required to give final recommendation as either - buy or not buy. Based only on the following information, "
            f"provide a single clear recommendation for {ticker}.\n\n"
            f"Trend: {trend}\n"
            f"Average Change: {change} percent\n"
            f"Volatility: {volatility} percent\n"
            f"News Highlights: {news_summary}\n\n"
            f"Output format: Recommendation is it buy? or not buy?. "
            f"Add stats from trend in the output. and recommendation : is it buy or not buy"
        )

        response = self.llm(prompt, max_new_tokens=50)[0]["generated_text"].strip()

        return response

# Test
summary_agent = SummaryAgent()
final_summary = summary_agent.generate_summary("AAPL", trend, change, volatility, news_summary)
print("Summary Agent Output")
print(final_summary)


Device set to use cpu


Summary Agent Output
buy


7.	Knowledge Embedding and Memory (MemoryAgent)
Summaries and insights are stored using sentence embeddings and efficient similarity search (FAISS index)
to maintain contextual memory. This allows future queries and expansions on past recommendations,
 supporting persistent and contextual financial advisory.


In [ ]:
class MemoryAgent:
    def __init__(self):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.index = faiss.IndexFlatL2(384)
        self.memory = {}

    def store_memory(self, ticker, text):
        embedding = self.embedder.encode([text])
        self.index.add(np.array(embedding, dtype="float32"))
        self.memory[ticker] = text

memory_agent = MemoryAgent()
memory_agent.store_memory("AAPL", final_summary)
print("Memory Agent Output for AAPL")
print(memory_agent.memory)


Memory Agent Output for AAPL
{'AAPL': 'buy'}


8.	Quality Evaluation (EvaluatorAgent)
The clarity and usefulness of each recommendation are assessed by an expert-tuned language model,
providing feedback on whether the output is actionable or needs refinement.
This meta-review step ensures higher reliability in deployment scenarios.


In [ ]:
class EvaluatorAgent:
    def __init__(self):
        self.llm = pipeline("text2text-generation", model="google/flan-t5-base")

    def evaluate(self, summary_text):
        prompt = (
            f"You are an expert financial analyst. Evaluate the clarity and usefulness "
            f"of the following stock recommendation in one concise line. "
            f"Provide suggestions if it is ambiguous.\n\n"
            f"Recommendation Text: {summary_text}\n\n"
            f"Output format: Yes, good recommendation  or Not, bad recommendation ."
        )

        response = self.llm(prompt, max_new_tokens=50)[0]["generated_text"].strip()
        return response

# Test Evaluator Agent
evaluator_agent = EvaluatorAgent()
evaluation = evaluator_agent.evaluate(final_summary)
print("Evaluator Agent Output")
print(evaluation)


Device set to use cpu


Evaluator Agent Output
Yes
